In [29]:
import pprint
import csv
import ais
import tensorflow as tf

pp = pprint.PrettyPrinter(indent=4)

In [37]:
# Fields
primaryFields = ['TIMESTAMP','TIME','MESSAGE_ID','MMSI','LAT','LONG','POS_ACCURACY','NAV_STATUS','SOG','COG','TRUE_HEADING','RAW_MESSAGE','FISHING_STATUS']

payloadFields = ['slot_increment', 'sync_state', 'true_heading', 'sog', 'slots_to_allocate', 'rot', 'nav_status', 'repeat_indicator', 'raim', 'id', 'spare', 'keep_flag', 'cog', 'timestamp', 'y', 'x', 'position_accuracy', 'rot_over_range', 'mmsi', 'special_manoeuvre', 'slot_timeout', 'slot_offset', 'slot_number', 'received_stations', 'utc_spare', 'utc_min', 'utc_hour', 'unit_flag', 'spare2', 'mode_flag', 'm22_flag', 'commstate_cs_fill', 'commstate_flag', 'display_flag', 'dsc_flag', 'band_flag', 'gnss', 'fix_type', 'type_and_cargo', 'spare3', 'dim_d', 'assigned_mode', 'dim_b', 'dim_c', 'dim_a', 'name', 'dte']

fieldnames = payloadFields + ['TIMESTAMP', 'FISHING_STATUS', 'F1', 'F2', 'F3', 'F4', 'F5']

In [38]:
# Open a new csv file to write out cleaned data
outputFile = open('data/clean.csv', 'wb')
writer = csv.DictWriter(outputFile, fieldnames=fieldnames)
writer.writeheader()

In [39]:
# Read file line by line
with open('data/training_data.csv') as inputFile:
    reader = csv.DictReader(inputFile)
    
    # 'row' is a dictionary of data, keyed by
    # the values in the first row of the csv file.
    for row in reader:

        rawMsg = row['RAW_MESSAGE']
        rawMsgElms = rawMsg.split(',')
        
        for i in range(len(rawMsgElms)) :
            # Add generic fields 1-5
            if i + 1 <= 5 :
                row['F' + str(i + 1)] = rawMsgElms[i]

            # Decode and add payload
            if i + 1 == 6 :
                decodedMsg = ais.decode(rawMsgElms[i], 0)
                for k in decodedMsg.keys() :
                    row[k] = decodedMsg[k]
            
            # Checksum using field 7
            if i + 1 == 7 :
                checksum = 0;
                m = rawMsg[1:-3]
                for j in range(len(m)) :
                    checksum = checksum ^ ord(m[j]);
                checksum = format(checksum, '02X')
                
                if checksum != rawMsgElms[i][2:] :
                    print('checksum did not match')
                    print(rawMsg)
        
        data = {}
        for k in fieldnames :
            if k in row :
                data[k] = row[k]
            else :
                data[k] = ''

        writer.writerow(data)
#         pp.pprint(row)

print('----')

----


In [40]:
outputFile.close()